In [ ]:
%cd /content/
!rm -rf DLProject/
!git clone https://github.com/ManuelaCorte/DLProject.git

In [ ]:
%cd /content/DLProject/

In [ ]:
%pip install ftfy regex tqdm spacy ultralytics
%pip install git+https://github.com/openai/CLIP.git
!python -m spacy download en_core_web_lg

In [ ]:
import sys
sys.path.append('DLProject/src/')

In [ ]:
import gdown
import os


# Download dataset and save under data/raw/ only if not already downloaded
url = "https://drive.google.com/uc?id=1xijq32XfEm6FPhUb7RsZYWHc2UuwVkiq"
if not os.path.exists("data/raw/refcocog.tar.gz"):
    print("Downloading dataset...")
    gdown.download(url=url, output="data/raw/", quiet=False, resume=True)
if not os.path.exists("data/raw/refcocog/"):
    print("Extracting dataset...")
    !tar -xf data/raw/refcocog.tar.gz -C data/raw/ --verbose

In [ ]:
%cd src/

In [ ]:
from tqdm import tqdm
from vgproject.utils.bbox_types import BboxType
from vgproject.utils.misc import custom_collate
from vgproject.data.dataset import VGDataset
from vgproject.data.data_types import Split
from vgproject.models.baseline import Baseline
from clip import clip
import torch
from torch.utils.data import DataLoader
from torchvision.ops import box_iou
from vgproject.utils.config import Config

cfg = Config.get_instance()

test_data = VGDataset(
    dir_path=cfg.dataset["path"],
    split=Split(cfg.dataset["split"]),
    output_bbox_type=BboxType.XYXY,
    transform_image=None,
    transform_text=clip.tokenize,
    dependencies=False,
)

dataloader = DataLoader(
    test_data,
    batch_size=cfg.model["batch_size"],
    shuffle=False,
    collate_fn=custom_collate,
    drop_last=True,
)

baseline = Baseline()

batches_acc = []
for batch, bboxes in tqdm(dataloader):
    prediction = baseline.predict(batch)
    bbox_pred = torch.stack([p.bounding_box for p in prediction]).to(baseline.device)
    bbox_gt = bboxes.clone().detach().squeeze(1).to(baseline.device)
    # print(bbox_pred.shape, bbox_gt.shape)
    iou = box_iou(bbox_pred, bbox_gt).to(baseline.device)
    acc = torch.mean(torch.diagonal(iou))
    batches_acc.append(acc)
    # print('Accuracy: ', acc)

accuracy = torch.mean(torch.stack(batches_acc))
print("Iou: ", accuracy)